<h1>Fermate autobus</h1>

Lo scopo principale del programma è quello di estrarre le informazioni relative alle fermate, come l'id della tratta, l'id della fermata, latitudine e longitudine, il nome della fermata, il numero dell'autobus e se si tratta di andata o ritorno.


Queste info vengono estratte dal file GTFS (General Transit Feed Specification), ovvero una collezione di file in formato CSV contenute in un file zip, che presentano tutti la stessa struttura, indipendentemente dalla regione di appartenenza.

Dai vari file, verranno estratti solo una serie di campi chiave, che verranno utilizzati per l'ontologia.

Librerie utilizzate:

In [1]:
import csv
import pandas as pd
import re
import numpy as np
import urllib
import requests
import time
import sys
import zipfile
import io

Apriamo direttamente il file .zip dall'URL, e preleviamo i file che ci servono, ovvero trips, stops e stop_times:

<i>Nella maggior parte dei file GTFS, i file al suo interno sono salvati con l'estensione .txt, ma in realtà sono formattati come dei CSV.</i>

In [2]:
url = 'https://opendata.comune.palermo.it/js/server/uploads/dataset/gtfs/_25062019120427.zip'
response = requests.get(url)
zf = zipfile.ZipFile(io.BytesIO(response.content))
try:
    trips = pd.read_csv(zf.open('trips.txt'))
    stops = pd.read_csv(zf.open('stops.txt'))
    stop_times = pd.read_csv(zf.open('stop_times.txt'))
except:
    trips = pd.read_csv(zf.open('trips.csv'))
    stops = pd.read_csv(zf.open('stops.csv'))
    stop_times = pd.read_csv(zf.open('stop_times.csv'))
    
df = pd.DataFrame()

Applymap(str) serve per convertire i DataFrame in stringa, in modo tale da facilitare i confronti sottostanti.

Analizziamo i vari file: <br>

1) Da trips, consideriamo i campi route_id, direction_id e trip_id:
- route_id indica il numero dell'autobus,
- direction_id identifica se è il percorso di andata o ritorno,
- trip_id è un codice univoco del viaggio;

2) Da stop_times, andiamo a trovare il trip_id uguale a quello del file trips, e consideriamo inoltre il campo stop_id;

3) Da stops, andiamo a ricercare il campo stop_id uguale a quello di stop_times, in modo da ottenere le coordinate delle varie fermate.


<i>f ed f2 sono dei DataFrame temporanei:</i>

In [3]:
trips = trips.applymap(str)
stops = stops.applymap(str)
stop_times = stop_times.applymap(str)

f = pd.merge(stops, stop_times, how='inner', on=['stop_id'])
f2 = pd.merge(trips, f, how='inner', on=['trip_id'])


df['IDVIAGGIO'] = f2['trip_id'].apply(
    lambda x: f2['trip_id'].str if x == f2['trip_id'].str else x)
df['IDFERMATA'] = f2['stop_id'].apply(
    lambda x: f2['stop_code'].str if x == f2['stop_id'].str else x)
df['LATITUDINEFERMATA'] = f2['stop_lat']
df['LONGITUDINEFERMATA'] = f2['stop_lon']
df['NOMEFERMATA'] = f2['stop_name']
df['NUMEROAUTOBUS'] = f2['route_id']

Trasformiamo il valore di direction_id (int) in un boolean:

<i>Non tutti i file GTFS presentano un campo direction_id: pertanto viene superato se non presente:</i>

In [4]:
try:
    # 0 -> andata, 1 -> ritorno
    df['ANDATA'] = f2['direction_id']
    df['ANDATA'].replace('0', 'True', inplace=True)
    df['ANDATA'].replace('1', 'False', inplace=True)
except:
    pass

Nei file sono presenti tutte le fermate di tutti i viaggi della giornata: dunque se un autobus è particolarmente frequente, esso si ripeterà più volte all'interno del file.

In questo caso, siamo interessato solamente alla tratta di andata e di ritorno, non a quante volte viene compiuta la tratta durante il giorno: di conseguenza, andiamo ad effettuare un drop di tutti i vari duplicati:

<i>Dato che non tutti i file presentano il campo direction_id, eliminiamo solamente i duplicati tramite il nome della fermata:</i>

In [5]:
try:
    df = df.drop_duplicates(subset=['NOMEFERMATA', 'ANDATA'], keep='first')
except:
    df = df.drop_duplicates(subset=['NOMEFERMATA'], keep='first')

Infine, salviamo il file.

In [6]:
df.to_csv('Output_autobus.csv', index=False, encoding='utf-8')

In [7]:
df

,IDVIAGGIO,IDFERMATA,LATITUDINEFERMATA,LONGITUDINEFERMATA,NOMEFERMATA,NUMEROAUTOBUS,ANDATA
0,19182_merged_87314,367,38.12563942,13.30097628,CASTELLANA - ROGAZIONISTI,534,False
1,19182_merged_87314,803,38.11738423,13.29119805,FALCONARA 2,534,False
2,19182_merged_87314,802,38.11749012,13.28986964,FALCONARA 3,534,False
3,19182_merged_87314,801,38.11666277,13.28881935,FALCONARA 4,534,False
4,19182_merged_87314,786,38.115533105,13.29093195,FALCONARA - FRANCESCO DA PICCIANO,534,False
5,19182_merged_87314,780,38.11839445,13.29752471,FALCONARA - AI VILLINI A S. ISIDORO,534,False
6,19182_merged_87314,782,38.11732335,13.29402536,FALCONARA - ANDREA DA BURGIO,534,False
7,19182_merged_87314,1734,38.12236424,13.30875109,ROCCAZZO - AMAT,534,False
8,19182_merged_87314,1736,38.12061268,13.30642862,ROCCAZZO - CASA DEL SOLE,534,False
9,19182_merged_87314,1908,38.12401355,13.30294937,SARULLO - PAPALEO,534,False


Il file <b>Output_autobus</b> contiene:
- Id univoco del viaggio,
- Id univoco della fermata, 
- Latitudine fermata, 
- Longitudine fermata,
- Nome della fermata, 
- Il nome dell'autobus che compie quel viaggio,
- Se il viaggio è di andata o ritorno